In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import string
import joblib
from scipy.sparse import hstack

# Load the dataset
df = pd.read_csv("spam.csv", encoding='latin-1')[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'text'})

# Convert labels to numerical format (ham -> 0, spam -> 1)
df['label'] = LabelEncoder().fit_transform(df['label'])

# Feature engineering: Extract basic non-NLP features
df['text_length'] = df['text'].apply(len)
df['num_digits'] = df['text'].apply(lambda x: sum(c.isdigit() for c in x))
df['num_uppercase'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()))
df['num_special_chars'] = df['text'].apply(lambda x: sum(1 for c in x if c in string.punctuation))

# Define feature set for numeric features
X_numeric = df[['text_length', 'num_digits', 'num_uppercase', 'num_special_chars']]
y = df['label']

# Vectorization: Convert text to numeric features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)  # Using raw counts
X_text = vectorizer.fit_transform(df['text'])

# Combine numeric features with text-based features for both models
X_combined = hstack([X_text, X_numeric])

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42, stratify=y)

# Train Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42, C=1.0, solver='liblinear')  # 'liblinear' for smaller datasets
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# Train Naïve Bayes model
nb_model = MultinomialNB(alpha=1.0)  # Laplace smoothing (alpha=1)
nb_model.fit(X_train.toarray(), y_train)  # Naive Bayes requires dense input
y_pred_nb = nb_model.predict(X_test.toarray())

# Evaluate models
print("Logistic Regression Results:")
print(classification_report(y_test, y_pred_lr))
print("Accuracy:", accuracy_score(y_test, y_pred_lr))

print("\nNaïve Bayes Results:")
print(classification_report(y_test, y_pred_nb))
print("Accuracy:", accuracy_score(y_test, y_pred_nb))

# Confusion Matrix
print("Logistic Regression Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))
print("Naïve Bayes Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))

# Save models
joblib.dump(lr_model, 'spam_classifier_lr.pkl')
joblib.dump(nb_model, 'spam_classifier_nb.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


Logistic Regression Results:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.99      0.92      0.95       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

Accuracy: 0.9883408071748879

Naïve Bayes Results:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       966
           1       0.81      0.92      0.86       149

    accuracy                           0.96      1115
   macro avg       0.90      0.94      0.92      1115
weighted avg       0.96      0.96      0.96      1115

Accuracy: 0.9596412556053812
Logistic Regression Confusion Matrix:
 [[965   1]
 [ 12 137]]
Naïve Bayes Confusion Matrix:
 [[933  33]
 [ 12 137]]


['vectorizer.pkl']